In [2]:
# Import core TensorFlow modules
import tensorflow as tf
import numpy as np

In [3]:
# Modules required for file download and extraction
import os
import sys
import tarfile
from six.moves.urllib.request import urlretrieve
from scipy import ndimage

In [4]:
def maybe_download(filename, url, force=False):
  """Download a file if not present."""
  if force or not os.path.exists('/root/pipeline/datasets/notmnist/' + filename):
    filename, _ = urlretrieve(url + filename, '/root/pipeline/datasets/notmnist/' + filename)
    print('\nDownload complete for {}'.format(filename))
  else:
    print('File {} already present.'.format(filename))
  return '/root/pipeline/datasets/notmnist/' + filename

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('{} already present - don\'t need to extract {}.'.format(root, filename))
  else:
    print('Extracting data for {}. This may take a while. Please wait.'.format(root))
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(root[0:root.rfind('/') + 1])
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  print(data_folders)
  return data_folders

In [5]:
# Locations to download data:
url = 'http://yaroslavvb.com/upload/notMNIST/'

In [39]:
# Download two datasets
train_zip_path = maybe_download('notMNIST_small.tar.gz', url)

File notMNIST_small.tar.gz already present.


In [40]:
# Extract datasets
train_folders = maybe_extract(train_zip_path)

/root/pipeline/datasets/notmnist/notMNIST_small already present - don't need to extract /root/pipeline/datasets/notmnist/notMNIST_small.tar.gz.
['/root/pipeline/datasets/notmnist/notMNIST_small/A', '/root/pipeline/datasets/notmnist/notMNIST_small/B', '/root/pipeline/datasets/notmnist/notMNIST_small/C', '/root/pipeline/datasets/notmnist/notMNIST_small/D', '/root/pipeline/datasets/notmnist/notMNIST_small/E', '/root/pipeline/datasets/notmnist/notMNIST_small/F', '/root/pipeline/datasets/notmnist/notMNIST_small/G', '/root/pipeline/datasets/notmnist/notMNIST_small/H', '/root/pipeline/datasets/notmnist/notMNIST_small/I', '/root/pipeline/datasets/notmnist/notMNIST_small/J']


In [9]:
len(train_folders)

10

In [38]:
image_height = 28  # Pixel height of images
image_width = 28  # Pixel width of images
pixel_depth = 255.0  # Number of levels per pixel
expected_img_shape = (image_height, image_width)  # Black and white image, no 3rd dimension
num_labels = len(train_folders)

def load_image_folder(folder):
  """Load the data for a single image label."""
  
  # Create a list of image paths inside the folder  
  image_files = os.listdir(folder)
  # Create empty numpy array to hold data
  dataset = np.ndarray(shape=(len(image_files), image_height, image_width),
                         dtype=np.float32)
  num_images = 0  # Counter for number of successful images loaded
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      # Read in image pixel data as floating point values
      image_data = ndimage.imread(image_file).astype(float)
      # Scale values: [0.0, 255.0] => [-1.0, 1.0] 
      image_data = (image_data - pixel_depth / 2) / (pixel_depth / 2)
      if image_data.shape != expected_img_shape:
        print('File {} has unexpected dimensions: '.format(str(image_data.shape)))
        continue
      # Add image to the numpy array dataset
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- skipping this file and moving on.')
  
  # Trim dataset to remove unused space
  dataset = dataset[0:num_images, :, :]
  return dataset

In [48]:
def make_data_label_arrays(num_rows, image_height, image_width):
  """
  Creates and returns empty numpy arrays for input data and labels
  """
  if num_rows:
    dataset = np.ndarray((num_rows, image_height, image_width), dtype=np.float32)
    labels = np.ndarray(num_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def collect_datasets(data_folders):
  datasets = []
  total_images = 0
  for label, data_folder in enumerate(data_folders):
    # Bring all test folder images in as numpy arrays
    dataset = load_image_folder(data_folder)
    num_images = len(dataset)
    total_images += num_images
    datasets.append((dataset, label, num_images))
  return datasets, total_images

def merge_train_validation_datasets(datasets, total_images, percent_validation):
    num_train = total_images * (1.0 - percent_validation)
    num_valid = total_images * percent_validation
    train_dataset, train_labels = make_data_label_arrays(num_train, image_height, image_width)
    valid_dataset, valid_labels = make_data_label_arrays(num_valid, image_height, image_width)
    
    train_counter = 0
    valid_counter = 0
    dataset_counter = 1
    for dataset, label, num_images in datasets:
      np.random.shuffle(dataset)
      if dataset_counter != len(datasets):
        n_v = num_images // (1.0 / percent_validation)
        n_t = num_images - n_v
      else:
        # Last label, make sure dataset sizes match up to what we created
        n_v = len(valid_dataset) - valid_counter
        n_t = len(train_dataset) - train_counter
      train_dataset[train_counter: train_counter + n_t] = dataset[:n_t]
      train_labels[train_counter: train_counter + n_t] = label
      valid_dataset[valid_counter: valid_counter + n_v] = dataset[n_t: n_t + n_v]
      valid_labels[valid_counter: valid_counter + n_v] = label
      train_counter += n_t
      valid_counter += n_v
      dataset_counter += 1
    return train_dataset, train_labels, valid_dataset, valid_labels

In [41]:
train_valid_datasets, train_valid_total_images = collect_datasets(train_folders)

('Could not read:', '/root/pipeline/datasets/notmnist/notMNIST_small/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png', ':', IOError('cannot identify image file',), '- skipping this file and moving on.')
('Could not read:', '/root/pipeline/datasets/notmnist/notMNIST_small/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png', ':', IOError('cannot identify image file',), '- skipping this file and moving on.')


In [45]:
train_dataset, train_labels, valid_dataset, valid_labels = \
  merge_train_validation_datasets(train_valid_datasets, train_valid_total_images, 0.1)

last one!


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:52: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:54: VisibleDeprecationWarning: using a non-int

In [47]:
len(train_dataset)

16851

In [ ]:
# Convert data examples into 3-D tensors
# 
def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels